# SQL Generation with Transformer API

In [ ]:
!pip install torch transformers bitsandbytes accelerate sqlparse

In [ ]:
import sys
!{sys.executable} -m pip install torch transformers bitsandbytes accelerate sqlparse

In [ ]:
import sys
!{sys.executable} -m pip uninstall -y torch torchvision torchaudio

In [ ]:
!{sys.executable} -m pip install --pre torch --index-url https://download.pytorch.org/whl/nightly/cu124


Looking in indexes: https://download.pytorch.org/whl/nightly/cu124
  Using cached https://download.pytorch.org/whl/nightly/cu124/torch-2.7.0.dev20250310%2Bcu124-cp311-cp311-win_amd64.whl.metadata (29 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 GB 94.5 MB/s eta 0:00:28
    --------------------------------------- 0.0/2.7 GB 105.9 MB/s eta 0:00:25
    --------------------------------------- 0.1/2.7 GB 109.7 MB/s eta 0:00:24
   - -------------------------------------- 0.1/2.7 GB 113.1 MB/s eta 0:00:23
   - -------------------------------------- 0.1/2.7 GB 112.7 MB/s eta 0:00:23
   - -------------------------------------- 0.1/2.7 GB 114.1 MB/s eta 0:00:23
   -- ------------------------------------- 0.2/2.7 GB 114.8 MB/s eta 0:00:22
   -- ------------------------------------- 0.2/2.7 GB 115.5 MB/s eta 0:00:22
   --- -----------------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!{sys.executable} -m pip install --pre torchvision --index-url https://download.pytorch.org/whl/nightly/cu124


Looking in indexes: https://download.pytorch.org/whl/nightly/cu124
  Using cached https://download.pytorch.org/whl/nightly/cu124/torchvision-0.22.0.dev20250226%2Bcu124-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.


ERROR: Could not find a version that satisfies the requirement torch==2.7.0.dev20250225+cu124 (from torchvision) (from versions: 2.7.0.dev20250310+cu124)
ERROR: No matching distribution found for torch==2.7.0.dev20250225+cu124


In [1]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

2.9.0+cu128
12.8
True
Tesla T4


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
torch.cuda.is_available()

True

In [4]:
available_memory = torch.cuda.get_device_properties(0).total_memory

In [5]:
print(available_memory)

15637086208


##Download the Model
Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16. If unavailable, use a GPU with minimum 8GB VRAM to load this in 8bit, or with minimum 5GB of VRAM to load in 4bit.

This step can take around 5 minutes the first time. So please be patient :)

In [6]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

##Set the Question & Prompt and Tokenize
Feel free to change the schema in the prompt below to your own schema

In [7]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

##Generate the SQL
This can be excruciatingly slow on a T4 in Colab, and can take 10-20 seconds per query. On faster GPUs, this will take ~1-2 seconds

Ideally, you should use `num_beams`=4 for best results. But because of memory constraints, we will stick to just 1 for now.

In [8]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [9]:
question = "What was our revenue by product in the New York region last month?"
generated_sql = generate_query(question)

In [10]:
print(generated_sql)


SELECT p.product_id,
       SUM(s.quantity * p.price) AS revenue
FROM sales s
JOIN salespeople sp ON s.salesperson_id = sp.salesperson_id
JOIN products p ON s.product_id = p.product_id
WHERE sp.region = 'New York'
  AND s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY p.product_id
ORDER BY revenue DESC NULLS LAST;


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

A. Include role definition

In [14]:
prompt_A = """### Role
You are a senior data engineer writing production-grade SQL.

### Task
Generate a syntactically correct and optimized SQL query.

### Constraints
- Use only tables and columns explicitly present in the schema.
- Use explicit JOIN syntax.
- Do not invent columns.
- Revenue = price * quantity.
- If the question cannot be answered using the schema, return exactly: I do not know
- Output SQL only. No explanations.

### Database Schema
{schema}

### Question
{question}

### SQL
"""


B. Explicit schema validation

In [16]:
prompt_B = """### Task
Generate a SQL query to answer the question.

### Before generating SQL:
1. List the exact tables needed.
2. List the exact columns needed.
3. Verify they exist in the schema.
4. If any table or column does NOT exist, return: I do not know

### Rules
- Do not invent columns.
- Revenue = price * quantity.
- Output ONLY the final SQL query.

### Schema
{schema}

### Question
{question}

### Answer
"""


Few-Shot Learning

In [17]:
prompt_C = """### Task
Generate a SQL query.

### Schema
{schema}

### Examples

Question: What is the total revenue by product?
SQL:
SELECT p.name, SUM(p.price * p.quantity) AS revenue
FROM products p
GROUP BY p.name;

Question: Which customer bought the most products?
SQL:
SELECT c.name, SUM(s.quantity) AS total_quantity
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
GROUP BY c.name
ORDER BY total_quantity DESC
LIMIT 1;

### Now answer:

Question: {question}
SQL:
"""


D. Review its own sql

In [18]:
prompt_D = """You are an expert SQL auditor.

Step 1: Generate a SQL query to answer the question.
Step 2: Check:
- Are all tables in the schema?
- Are all columns in the schema?
- Is revenue computed as price * quantity?
- Does the query logically answer the question?

If any check fails, return exactly: I do not know

If all checks pass, output ONLY the final SQL query.

### Schema
{schema}

### Question
{question}

### Final SQL
"""


prompt evaluation

In [19]:
# Extract schema automatically from original prompt
schema = prompt.split("### Database Schema")[-1]

In [25]:
prompts = {
    "A_engineer": prompt_A,
    "B_validator": prompt_B,
    "C_fewshot": prompt_C,
    "D_selfcritic": prompt_D
}

test_questions = {
    "simple_aggregation": "What is the total revenue by product?",
    "join_query": "Which customer generated the highest revenue?",
    "impossible_column": "What is the average delivery time?",
    "impossible_concept": "What was customer churn last quarter?"
}

def generate_with_prompt(prompt_template, question, schema):
    updated_prompt = prompt_template.format(question=question, schema=schema)

    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")

    generated_ids = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=False,
        num_beams=1,
    )

    output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()

    return output


In [26]:
import re

def extract_columns_from_schema(schema_text):
    columns = re.findall(r"\n\s+([a-zA-Z_]+)\s", schema_text)
    return list(set([col.lower() for col in columns]))

schema_columns = extract_columns_from_schema(schema)

schema_columns


['quantity',
 'address',
 'sale_id',
 'price',
 'supply_price',
 'customer_id',
 'create',
 'salesperson_id',
 'name',
 'sale_date',
 'supplier_id',
 'product_id',
 'region']

In [27]:
def detect_unknown_columns(sql, schema_columns):
    tokens = re.findall(r"\b[a-zA-Z_]+\b", sql.lower())
    unknown = []

    for token in tokens:
        if token in schema_columns:
            continue
        if token in ["select", "from", "join", "where", "group", "by",
                     "order", "limit", "sum", "count", "as", "on",
                     "desc", "asc"]:
            continue
        if token.endswith("_id"):
            continue

        if token not in schema_columns:
            unknown.append(token)

    return list(set(unknown))


In [28]:
# checks to each question

def check_revenue_formula(sql):
    return "price * quantity" in sql.lower()

def check_idk_behavior(question_key, sql):
    if "impossible" in question_key:
        return "i do not know" in sql.lower()
    return True


In [29]:
results = []

for prompt_name, prompt_template in prompts.items():
    for question_key, question in test_questions.items():

        sql = generate_with_prompt(prompt_template, question, schema)

        unknown_cols = detect_unknown_columns(sql, schema_columns)
        revenue_ok = check_revenue_formula(sql)
        idk_ok = check_idk_behavior(question_key, sql)

        results.append({
            "prompt": prompt_name,
            "question": question_key,
            "unknown_columns": unknown_cols,
            "revenue_correct": revenue_ok,
            "idk_correct": idk_ok
        })



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [30]:
import pandas as pd

df_results = pd.DataFrame(results)
df_results


,prompt,question,unknown_columns,revenue_correct,idk_correct
0,A_engineer,simple_aggregation,"[answers, unit, table, generate, supplier, dat...",True,True
1,A_engineer,join_query,"[answers, unit, table, generate, supplier, dat...",True,True
2,A_engineer,impossible_column,"[answers, unit, table, generate, supplier, dat...",True,True
3,A_engineer,impossible_concept,"[answers, unit, table, generate, churn, suppli...",True,True
4,B_validator,simple_aggregation,"[answers, unit, table, generate, supplier, dat...",True,True
5,B_validator,join_query,"[answers, unit, table, generate, supplier, dat...",True,True
6,B_validator,impossible_column,"[answers, unit, table, generate, supplier, dat...",True,True
7,B_validator,impossible_concept,"[answers, unit, table, generate, churn, suppli...",True,True
8,C_fewshot,simple_aggregation,"[answers, total, represented, unit, table, gen...",False,True
9,C_fewshot,join_query,"[answers, total, represented, unit, table, gen...",False,True


In [31]:
results.append({
    "prompt": prompt_name,
    "question": question_key,
    "sql_output": sql,
    "unknown_columns": unknown_cols,
    "revenue_correct": revenue_ok,
    "idk_correct": idk_ok
})

Findings

The results show that role-based and constraint-driven prompts (A, B, D) produced consistent and logically correct SQL, respecting the revenue formula and properly returning “I do not know” for unsupported queries. In contrast, the few-shot prompt (C) degraded performance, frequently ignoring explicit rules and failing on impossible questions. This suggests structured constraints outperform example-driven prompting for specialized SQL models.